# Proyecto EverPeak

### 1. Cargar el dataset y revisar estructura general

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv("/datasets/everpeak_retail.csv")

### Hacer un conteo de valores faltantes por columna:

payment_missing = df["payment_method"].isna().sum() 
city_missing = df["city"].isna().sum() 
state_missing = df["state"].isna().sum() 

print("payment_method missing:", payment_missing)
print("city missing:", city_missing)
print("state missing:", state_missing)


### Medir cuántos registros tienen fechas sospechosas y montos anormalmente altos:

Formatear la columna que contiene fechas a formato datetime:
df["order_date"] = pd.to_datetime(df["order_date"], errors="coerce") 

invalid_year_2026_count = (df["order_date"].dt.year == 2026).sum() 
missing_order_date_count = df["order_date"].isna().sum() 

print("order_date año 2026:", invalid_year_2026_count)
print("order_date missing:", missing_order_date_count)


### Calcular la cardinalidad de columnas clave para entender si son IDs, categorías o variables poco útiles.

customer_id_unicos = df["customer_id"].nunique() 
payment_unicos = df["payment_method"].nunique()
city_unicos = df["city"].nunique()
state_unicos = df["state"].nunique()

print("customer_id nunique:", customer_id_unicos)
print("payment_method nunique:", payment_unicos)
print("city nunique:", city_unicos)
print("state nunique:", state_unicos)

### Detectar patrones de missingness por grupo:

missing_city_by_pay = df["city"].isna().groupby(df["payment_method"]).mean()
print(missing_city_by_pay)


### Medir cómo cambia la métrica cuando imputas o eliminas datos.

before = df["order_value"].mean()  # esta es la media original
df["order_value"]= df["order_value"].dropna()
df["order_value_imputed"] = df["order_value"].fillna(df["order_value"].median())   # para imputar
after = df["order_value_imputed"].mean() # la media después de imputar

print(before)
print(after)

### Comparar tres versiones de una columna numérica para decidir la estrategia final.

#### Creamos "before"
before = df["customer_age"].mean() 

#### Creamos "customer_age_med"
df["customer_age_med"] = df["customer_age"].fillna(df["customer_age"].median())
after_med = df["customer_age_med"].mean()

#### Creamos "customer_age_mean"
df["customer_age_mean"] = df["customer_age"].fillna(df["customer_age"].mean()) 
after_mean = df["customer_age_mean"].mean()

print(before)
print(after_med)
print(after_mean)


### Aplicar una misma transformación a varias columnas usando una función + un bucle interno.

#### Usamos la función convertir_columnas_numericas:
def convertir_columnas_numericas(df, columnas):
    for col in columnas:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

#### 1. Crear la lista de columnas numéricas
columnas_numericas = ["price", "quantity", "order_value"] 

#### 2. Aplicar la función convertir_columnas_numericas
df = convertir_columnas_numericas (df, columnas_numericas) 

#### 3. Revisar resultado
df.info()


### Demostrar la escalabilidad del enfoque:

#### Usar la función convertir_columnas_numericas:
def convertir_columnas_numericas(df, columnas):
    for col in columnas:
        df[col] = pd.to_numeric(df[col], errors="coerce")
    return df

columnas_numericas = ["price", "quantity", "order_value"]

#### 1. Agregar la nueva columna
columnas_numericas.append ("customer_id")

#### 2. Aplicar nuevamente la función
df = convertir_columnas_numericas(df, columnas_numericas)

df.info()

### Función que limpie varias columnas categóricas al mismo tiempo:

def step_strip_text(df):
    columnas = ["product_category", "city", "state"]#  Estas son las columnas categóricas

    # Loop para aplicar strip() a cada columna
    for col in columnas:
        df[col]= df[col].str.strip()

    # Regresar DF limpio
    return df

#### Probar
df = step_strip_text(df)
print(df.head())


### Función para que pueda trabajar con múltiples columnas numéricas usando un bucle.

#### Función mejorada
def reemplazar_sentinels(df, sentinels, numeric_cols):
    for col in numeric_cols:
        df[col]= df[col].replace(sentinels, pd.NA) 
    return df

#### Valores ausentes iniciales
print("Valores ausentes iniciales:")
print(df[["customer_age", "price"]].isna().sum())

#### Valores a corregir y columnas
valores_erroneos = [-999, 999, 0, -1]
columnas_numericas = ["customer_age", "price"]

#### Aplicar función y observar cambios
df = reemplazar_sentinels(df, valores_erroneos, columnas_numericas) 
print("\nValores ausentes después:")
print(df[["customer_age", "price"]].isna().sum())



### Función para rellenar ausentes

#### Crear función
def rellenar_ausentes(df, cols_fill):
    for col in cols_fill:
        df[col] = pd.to_numeric(df[col], errors="coerce") 
        df[col].fillna(df[col].mean(), inplace=True)
    return df

#### Valores ausentes iniciales
print("Valores ausentes iniciales:")
print(df[["customer_age", "price"]].isna().sum())

#### Definir columnas a rellenar
columnas_rellenar = ["customer_age","price" ]

#### Aplicar función y observar cambios
df = rellenar_ausentes(df, columnas_rellenar)
print("\nValores ausentes después:")
print(df[["customer_age", "price"]].isna().sum())

### Crear Pipeline de limpieza

#### Funciones auxiliares
   
def reemplazar_sentinels(df, sentinels, numeric_cols):
    for col in numeric_cols:
        df[col] = df[col].replace(sentinels, pd.NA)
    return df

def rellenar_ausentes(df, cols_fill):
    for col in cols_fill:
        df[col] = pd.to_numeric(df[col], errors='coerce')
        df[col].fillna(df[col].mean(), inplace=True)
    return df

#### Crear función pipeline
def limpiar_df(df):
    valores_erroneos=[-999,999,0,-1] 
    columnas_numericas=["customer_age","price"] 
    
    df = reemplazar_sentinels(df, valores_erroneos, columnas_numericas) 
    df = rellenar_ausentes(df, columnas_numericas)
    
    return df

#### observar valores ausentes iniciales
print("Valores ausentes iniciales:")
print(df[["customer_age", "price"]].isna().sum())

#### Aplicar pipeline completo
df = limpiar_df(df)
print("\nValores ausentes después del pipeline:")
print(df[["customer_age", "price"]].isna().sum())



### Contestando al equipo de Inteligencia Comercial, necesitan una visión rápida del desempeño de las categorías Fashion y Sports para entender su situación actual y detectar posibles anomalías en los datos.

#### Crear dataframes para cada categoría
df_fashion = df[df["product_category"]=="Fashion"]
df_sports = df[df["product_category"]=="Sports"]

#### Resumen de columnas numéricas con describe()
columnas_numericas = ['order_value', 'customer_age', 'price', 'quantity']

print('Resumen estadístico de la categoría Fashion')
print(df_fashion[columnas_numericas].describe())

print()#Salto de Línea
print('Resumen estadístico de la categoría Sports')
print(df_sports[columnas_numericas].describe())

### Ayudando al equipo que quiere entender si el gasto típico de los clientes que compran productos de supermercado está siendo afectado por outliers. 

#### Filtrar la categoría Grocery
df_grocery = df[df["product_category"]=="Grocery"]

#### Calcular media y mediana del gasto
promedio = df_grocery["order_value"].mean()
mediana = df_grocery["order_value"].median()

#### Mostrar resultados
print("Promedio del gasto en Grocery:", promedio)
print("Mediana del gasto en Grocery:", mediana)

# Interpretación según comparación de media y mediana
print("El promedio está afectado por outliers o valores atípicos en Grocery.")


### Respondiendo al equipo la sospecha que existen outliers en la cantidad de productos comprados. 

#### Promedio y mediana de quantity
print("Promedio de quantity: ", df["quantity"].mean())
print("Mediana de quantity: ", df["quantity"].median())
print("El promedio está afectado por los outliers o valores atípicos.")


### Resumen numérico por ciudad para el equipo que necesita una visión rápida del comportamiento de los clientes en las ciudades New York y Los Angeles, para comprender su situación actual y detectar posibles anomalías en los datos.

#### Crear dataframes para cada categoría
df_ny = df[df["city"]=="New York"]   
df_la = df[df["city"]=="Los Angeles"] 

#### --- Resumen de columnas numéricas
columnas_numericas = ['order_value', 'customer_age', 'price', 'quantity']

print('Resumen estadístico de la ciudad New York')
print(df_ny[columnas_numericas].describe()) 

print()#Salto de Línea
print('Resumen estadístico de la ciudad Los Angeles')
print(df_la[columnas_numericas].describe()) 
    

### Ayudando al equipo que necesita entender la distribución de los métodos de pago y las categorías de producto en Nueva York y Chicago para identificar patrones dominantes, diversidad de categorías y posibles anomalías.

####  Columnas categóricas
columnas_categoricas = ['payment_method', 'product_category']

#### Filtra por ciudad
df_ny = df[df["city"]=="New York"] 
df_chicago = df[df["city"]=="Chicago"] 

#### Resumen categórico New York
print("Resumen categórico - New York")
print(df_ny[columnas_categoricas].describe())

print() # salto de línea

#### Resumen categórico Chicago
print("Resumen categórico - Chicago")
print(df_chicago[columnas_categoricas].describe()

### Revisión de todas las categorías de una columna junto con su frecuencia y porcentajes.

columnas_categoricas = ['product_category', 'payment_method', 'city', 'state']

#### Distribución completa de columnas categóricas
for col in columnas_categoricas: 
    print(col)
    print("Frecuencia absoluta")
    print(df[col].value_counts())  
    print("Frecuencia relativa")
    print(df[col].value_counts(normalize=True) ) 
    print() # mantén salto de línea

### Ayudando al equipo que necesita entender la distribución de los métodos de pago y las ciudades asociadas a la categoría Toys, para identificar patrones dominantes, diversidad de ciudades y posibles anomalías.

#### Columnas categóricas
columnas_categoricas = ['payment_method', 'city']

#### Filtra por categoría
df_toys = df[df["product_category"]=="Toys"] 

#### Resumen categórico Toys
print("Resumen categórico - Toys")
print(df_toys[columnas_categoricas].describe() )    



### Revisión de todas las ciudades asociadas a la categoría Sports, mostrando su frecuencia absoluta y su frecuencia relativa para entender la distribución de ubicaciones dentro de esta categoría.

#### Filtra por categoría
df_sports = df[df["product_category"] =="Sports"]

#### Distribución de city
print("Frecuencia absoluta")
print(df_sports["city"].value_counts() ) 
print("\nFrecuencia relativa")
print(df_sports["city"].value_counts(normalize=True))

### Ayudando al equipo de Inteligencia Comercial que quiere entender cómo se distribuyen los precios y detectar posibles valores atípicos que puedan afectar los análisis.

#### Graficar histograma
counts, bin_edges, _ = plt.hist(df["price"], bins=10, range=(0,1000), color="skyblue", edgecolor="black")

#### Mostrar las marcas de los bins en el eje X
plt.xticks(bin_edges) 

#### Etiquetas y título del gráfico
plt.xlabel("Precio") 
plt.ylabel("Cantidad") 
plt.title("Distribución de Precios") 
plt.show()

### Ayudar al equipo que quiere entender la distribución de las edades de los clientes (customer_age) para segmentar mejor las estrategias de marketing.

#### Graficar histograma
counts, bin_edges, _ = plt.hist(df["customer_age"], bins=10, color="skyblue", edgecolor="black")

#### Mostrar las marcas de los bins en el eje X
plt.xticks(bin_edges)

#### Etiquetas y título del gráfico
plt.xlabel("Edades de los Clientes")
plt.ylabel("Cantidad") 
plt.title("Distribución de Edades") 
plt.show()

### Ayudar al equipo de Inteligencia Comercial que quiere entender cómo se distribuyen los precios y detectar posibles valores atípicos que puedan afectar los análisis.

#### Graficar histograma
counts, bin_edges, _ = plt.hist(df["price"], bins=10, range=(0,1000), color="skyblue", edgecolor="black")

#### Mostrar las marcas de los bins en el eje X
plt.xticks(bin_edges)

#### Etiquetas y título del gráfico
plt.xlabel("Precio") 
plt.ylabel("Cantidad") 
plt.title("Distribución de Precios")
plt.show()


### Ayudar al equipo de Inteligencia Comercial que necesita analizar cómo se distribuyen las cantidades compradas (quantity) dentro de la categoría "Toys", para detectar valores extremos que podrían afectar decisiones de inventario y logística.

#### Graficar BoxPlot
sns.boxplot(x=df_toys["quantity"], color="skyblue" )

#### Etiquetas y título del gráfico
plt.title("Boxplot de Cantidades - Categoría Toys" )
plt.xlabel("Cantidad Comprada")
plt.show()


### Ayudar al equipo que quiere entender la distribución de las edades de los clientes (customer_age) para segmentar mejor las estrategias de marketing.

#### Graficar histograma
counts, bin_edges, _ = plt.hist(df["customer_age"], bins=10, color="skyblue", edgecolor="black")

#### Mostrar las marcas de los bins en el eje X
plt.xticks(bin_edges)

#### Etiquetas y título del gráfico
plt.xlabel("Edades de los Clientes")
plt.ylabel("Cantidad") 
plt.title("Distribución de Edades")
plt.show()

### Ayudar al equipo que necesita analizar cuánto gastan los clientes en las categorías “Fashion” y “Sports”, evaluando la dispersión del order_value y detectando posibles outliers que puedan afectar decisiones de negocio.

#### Filtrar datos
df_fashion =df[df["product_category"]=="Fashion"] 
df_sports =df[df["product_category"]=="Sports"] 

#### BoxPlot Categoría Fashion
sns.boxplot(x=df_fashion["order_value"], color="skyblue")
plt.xlabel("Gasto del Cliente") 
plt.title("Boxplot de Gastos - Categoría Fashion")
plt.show()

#### BoxPlot Categoría Sports
sns.boxplot(x=df_sports["order_value"], color="skyblue")
plt.xlabel("Gasto del Cliente") 
plt.title("Boxplot de Gastos - Categoría Sports")
plt.show()

### Identificar outliers en order_value usando IQR.

#### calcular Q1, Q3 e IQR
Q1 = df["order_value"].quantile(0.25) 
Q3 = df["order_value"].quantile(0.75) 
IQR = Q3 - Q1 

#### calcular límite inferior y superior
limite_inferior = Q1-1.5*IQR 
limite_superior = Q3+1.5*IQR 

#### Mostrar resultados
print('Primer cuartil: ', Q1)
print('Tercer cuartil: ', Q3)
print('IQR: ', IQR)

print("\nRegistros abajo del límite inferior")
print(df[df["order_value"] < limite_inferior ])

print("\nRegistros arriba del límite superior")
print(df[df["order_value"] > limite_superior ])


### Identificar outliers en order_value usando Z-Score.

#### cálculo de la media
mean = df["order_value"].mean() 

#### cálculo de la desviación estándar
std = df["order_value"].std() 

#### Crea el z score
df['z'] = (df["order_value"] - mean) / std 

#### Calcula los valores extremos
print(df[df["z"].abs()>3])

### Comparar métodos IQR y Z-Score en la columna price. 

#### calcular Q1, Q3 e IQR
Q1 = df["price"].quantile(0.25)
Q3 = df["price"].quantile(0.75)
IQR = Q3-Q1 

#### calcular límite inferior y superior
lower = Q1 - 1.5 * IQR
upper = Q3 + 1.5 * IQR

#### calcular media, desviación estándar y z-score
mean = df["price"].mean() 
std = df["price"].std() 
df['z'] = (df["price"]-mean)/std  

#### Resultados
print('Outliers usando IQR:')
print(df[(df["price"]<lower) | (df["price"]>upper)])

print('\nOutliers usando Z-Score:')
print(df[df["z"].abs()>3])


### Respondiendo a la gerencia de marketing que nos ha pedido crear una regla que indique si los clientes en general superan cierto volumen de compra.

#### Calcular promedio y mediana
cantidad_promedio = df["quantity"].mean() 
cantidad_mediana = df["quantity"].median()
print("Promedio:", cantidad_promedio)
print("Mediana:",cantidad_mediana)
print() # salto de línea

#### Segmentación con media o promedio
if cantidad_promedio > 22: 
	print("En promedio: volumen alto")
else: 
	print("En promedio: volumen bajo" )
	
#### Segmentación con mediana
if cantidad_mediana > 22: 
    print("Según la mediana: volumen alto")
else:
    print("Según la mediana: volumen bajo")

### La gerencia de marketing nos ha pedido mejorar la regla anterior agregando una tercera categoría.

#### Calcular promedio y mediana
cantidad_promedio = df['quantity'].mean()
cantidad_mediana = df['quantity'].median()
print("Promedio:", cantidad_promedio)
print("Mediana:",cantidad_mediana)
print()

#### Segmentación con media o promedio
if cantidad_promedio > 22:
	print("En promedio: volumen alto")
elif cantidad_promedio >= 10: 
    print("En promedio: volumen medio")
else: 
	print("En promedio: volumen bajo")
	
#### Segmentación con mediana
if cantidad_mediana > 22:
	print("Según la mediana: volumen alto")
elif cantidad_mediana >= 10:
    print("Según la mediana: volumen medio")
else: 
	print("Según la mediana: volumen bajo")

### La gerencia de marketing nos ha pedido segmentar a los clientes por volumen de compra.

#### Crear columna nueva
df["volume_segment"] = np.where(
    (df["quantity"]>22),"High Volume", 
    "Low Volume")  

#### Verificar cambios
print(df['volume_segment'].value_counts())

### La gerencia de marketing nos ha pedido segmentar los clientes por edad y por volumen de compra, separando en 4 segmentos.

def classify_volume(row):
    age = row["customer_age"] 
    qty = row["quantity"] 

    # 1. Manejo de valores nulos/faltantes
    if pd.isna(age) or pd.isna(qty):
        return "Error en Datos"

    # 2. Segmentación de Altas Cantidades
    if qty > 22:
        if age > 55: 
            return "Sr. High Volume" 
        else:  
            return "Jr. High Volume"
            
    # 3. Segmentación Bajas Cantidades
    elif qty <= 22: 
        if age > 55: 
            return "Sr. Low Volume" 
        else: 
            return "Jr. Low Volume" 

#### aplicar función y verificar cambios
df["volume_segment"] = df.apply(classify_volume, axis=1 )
print(df['volume_segment'].value_counts())

### La gerencia de finanzas nos ha solicitado segmentar las transacciones según el método de pago y el volumen de compra, para analizar patrones de pago en compras altas y bajas.

def classify_payment(row):
    card = row["payment_method"] 
    qty = row["quantity"] 

    # 1. Manejo de valores nulos/faltantes
    if pd.isna(card) or pd.isna(qty):
        return "Error en Datos"

    # 2. Segmentación de Altas Cantidades
    if qty > 22:
        if card == "credit_card" or card == "debit_card":
            return "card_high_volume"
        else: 
            return "no_card_high_volume" 
            
    # 3. Segmentación Bajas Cantidades
    elif qty <= 22:
        if card == "credit_card" or card == "debit_card":
            return "card_low_volume" 
        else: 
            return "no_card_low_volume"

#### aplicar función y verificar cambios
df['payment_segment'] = df.apply(classify_payment, axis=1) 
print(df['payment_segment'].value_counts())